In [4]:
import sys
import mysql.connector
import xlsxwriter
import pandas as pd
import sys
import mysql.connector
import numpy as np
import os
from dotenv import load_dotenv
load_dotenv()
id=191
# id=str(sys.argv[1])
#configurar la conexion a la base de datos
DB_USERNAME = os.getenv('DB_USERNAME')
DB_DATABASE = os.getenv('DB_DATABASE')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_PORT = os.getenv('DB_PORT')

a_color='#354F84'
a_lite='#b4c7ed'
b_color='#91959E'
# Conectar a DB
cnx = mysql.connector.connect(user=DB_USERNAME,
                              password=DB_PASSWORD,
                              host='localhost',
                              port=DB_PORT,
                              database=DB_DATABASE,
                              use_pure=False)
#Seccion para traer informacion de la base
# join para cobros
quotation=pd.read_sql("select * from quotations where id=" +str(id),cnx)

writer = pd.ExcelWriter('storage/report/administrativo'+str(id)+'.xlsx', engine='xlsxwriter')

workbook = writer.book
##FORMATOS PARA EL TITULO------------------------------------------------------------------------------
rojo_l = workbook.add_format({
    'bold': 0,
    'border': 0,
    'align': 'center',
    'valign': 'vcenter',
    #'fg_color': 'yellow',
    'font_color': 'red',
    'font_size':16})
negro_s = workbook.add_format({
    'bold': 0,
    'border': 0,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'font_size':12})
negro_b = workbook.add_format({
    'bold': 2,
    'border': 0,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'font_size':13,
    
    'text_wrap': True,
    'num_format': 'dd/mm/yyyy'}) 
rojo_b = workbook.add_format({
    'bold': 2,
    'border': 0,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'red',
    'font_size':13})      

#FORMATOS PARA CABECERAS DE TABLA --------------------------------
header_format = workbook.add_format({
    'bold': True,
    'text_wrap': True,
    'valign': 'center',
    'fg_color': 'yellow',
    'border': 1,})

blue_header_format = workbook.add_format({
    'bold': True,
    'bg_color': a_color,
    'text_wrap': True,
    'valign': 'top',
    'align': 'center',
    'border_color':'white',
    'font_color': 'white',
    'border': 1})
blue_header_format_bold = workbook.add_format({
    'bold': True,
    'bg_color': a_color,
    'text_wrap': True,
    'valign': 'top',
    'align': 'center',
    'border_color':'white',
    'font_color': 'white',
    'num_format': '[$$-409]#,##0.00',
    'border': 1,
    'font_size':13})
blue_footer_format_bold = workbook.add_format({
    'bold': True,
    'bg_color': a_color,
    'text_wrap': True,
    'valign': 'top',
    'align': 'center',
    'border_color':'white',
    'font_color': 'white',
    'border': 1,
    'num_format': '[$$-409]#,##0.00',
    'font_size':11})
blue_footer_format_bold_kg = workbook.add_format({
    'bold': True,
    'bg_color': a_color,
    'text_wrap': True,
    'valign': 'top',
    'align': 'center',
    'border_color':'white',
    'font_color': 'white',
    'border': 1,
    'num_format': '#,##0.00',
    'font_size':11})

red_header_format = workbook.add_format({
    'bold': True,
    'bg_color': b_color,
    'text_wrap': True,
    'valign': 'top',
    'align': 'center',
    'border_color':'white',
    'font_color': 'white',
    'border': 1})

red_header_format_bold = workbook.add_format({
    'bold': True,
    'bg_color': b_color,
    'text_wrap': True,
    'valign': 'top',
    'align': 'center',
    'border_color':'white',
    'font_color': 'white',
    'border': 1,
    'font_size':13})


#FORMATOS PARA TABLAS PER CE------------------------------------

blue_content = workbook.add_format({
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    
    'border_color':a_color,
    'font_size':10,
    'num_format': '[$$-409]#,##0.00'})
blue_content_unit = workbook.add_format({
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    
    'border_color':a_color,
    'font_size':10,
    'num_format': '0.00'})
blue_content_lite = workbook.add_format({
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'bg_color':a_lite,
    'border_color':a_color,
    'font_size':10,
    'num_format': '[$$-409]#,##0.00'})
blue_content_unit_lite = workbook.add_format({
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'bg_color':a_lite,
    'border_color':a_color,
    'font_size':10,
    'num_format': '0.00'})
blue_content_bold = workbook.add_format({
    'bold': True,
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'font_size':11,
    'border_color':a_color,
    'num_format': '[$$-409]#,##0.00'})

blue_content_bold_dll = workbook.add_format({
    'bold': True,
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'font_size':11,
    'bg_color': '#b4e3b1',
    'border_color':a_color,
    'num_format': '[$$-409]#,##0.00'})
blue_content_date = workbook.add_format({
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'font_size':9,
    'border_color':a_color,
    'num_format': 'dd/mm/yyyy'})
#FOOTER FORMATS---------------------------------------------------------
observaciones_format = workbook.add_format({
    'bold': True,
    'text_wrap': True,
    'valign': 'top',
    'fg_color':'#BDD7EE',
    'border': 1})

blue_content_dll = workbook.add_format({
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'bg_color': '#b4e3b1',
    'border_color':a_color,
    'font_size':10,
    'num_format': '[$$-409]#,##0.00'})

total_cereza_format = workbook.add_format({
    'bold': True,
    'text_wrap': True,
    'valign': 'top',
    'fg_color':'#F4B084',
    'border': 1})
df=quotation[['id','user_id']]

df[0:1].to_excel(writer, sheet_name='Sheet1', startrow=7,startcol=6, header=False, index=False)
worksheet = writer.sheets['Sheet1']
#Encabezado del documento--------------

import datetime

currentDateTime = datetime.datetime.now()
date = currentDateTime.date()
year = date.strftime("%Y")

from tablas_dict import tablas
aceros=pd.read_sql('select * from steels ',cnx)
aceros.loc[aceros['caliber']=='EST 3 IN','caliber']='EST3'

products=pd.DataFrame()
#iterar sobre tablas
for i in tablas:
    print(i)
    #buscar en la base de datos todos los productos de esta tabla
    #pertenecientes a la cotizacion pedida por el usuario.
    p=pd.read_sql('select * from '+i+' where quotation_id = '+str(id),cnx)
    p=p.assign(tabla=i)
    if(('cost' not in p.columns)&(len(p)>0)):
        if('caliber' not in p.columns):
             #esto es en especifico por un caso en que todas kas piezas son cal 14
             p=p.assign(caliber='14')
        try:
            p['caliber']=p['caliber'].str.replace('-','')
        except:
            print(' ')
        print(str(p['caliber'].values[0]))
        costo=aceros.loc[aceros['caliber']==str(p['caliber'].values[0]),'cost'].values[0]
        if('total_kg' in p.columns):
            p=p.assign(cost=costo*p.total_kg)
        if('total_weight' in p.columns):
           
            p=p.assign(cost=costo*p.total_weight)
        if('weight_kg' in p.columns):
         
            p=p.assign(cost=costo*p.weight_kg)
        if('weight' in p.columns):
          
            p=p.assign(cost=costo*p.weight)
        if('long' in p.columns):
              
            p=p.assign(cost=costo*p.long)

        print(i)
    products=products.append(p,ignore_index=True)

products=products.loc[products['amount']>0].reset_index(drop=True)
cols_to_fill_str=['description','protector','model','sku']
products[cols_to_fill_str]=products[cols_to_fill_str].fillna('')
cols_kg=['weight','total_kg','total_weight','weight_kg']
cols_m2=['m2','total_m2']
price_cols=['price','total_price','import','unit_price']
products[cols_kg+cols_m2+price_cols+['amount']]=products[cols_kg+cols_m2+price_cols+['amount']].fillna(0)
largo_cols=['long','length','length_meters','frame_background',
       'length_dimension', 'dimensions']
ancho_cols=['uncut_front',  'uncut_background',
       'depth']

pricelist_protectors=pd.read_sql('select * from price_list_protectors',cnx)
quotation_protectors=pd.read_sql('select quotation_protectors.*, protectors.sku from quotation_protectors  inner join protectors on protectors.protector=quotation_protectors.protector where quotation_id ='+str(id),cnx)
quotation_shlf=pd.read_sql('select * from selective_heavy_load_frames where quotation_id ='+str(id),cnx)
df[0:1].to_excel(writer, sheet_name='Sheet1', startrow=7,startcol=6, header=False, index=False)
materials=pd.read_sql('select * from (materials left join price_list_screws on materials.price_list_screw_id= price_list_screws.id)left join price_lists on price_lists.id=materials.price_list_id',cnx)
materials['type']=materials['type'].fillna('')
worksheet = writer.sheets['Sheet1']
#Encabezado del documento--------------
worksheet.merge_range('B2:F2', 'REPORTE POR COTIZACION ', negro_b)
worksheet.merge_range('B3:F3', 'ADMINISTRATIVO', negro_s)
worksheet.merge_range('B4:F4', 'COSTOS ', negro_b)
worksheet.write('H2', 'AÑO', negro_b)

worksheet.write('I2', year, negro_b)
worksheet.merge_range('J2:K3', """FECHA DEL REPORTE
DD/MM/AAAA""", negro_b)

worksheet.write('L2', date, negro_b)
worksheet.insert_image("A1", "img/logo/logo.png",{"x_scale": 0.6, "y_scale": 0.6})
worksheet.merge_range('A6:A8', 'PDA', blue_header_format)
worksheet.merge_range('B6:B8', 'SKU	', blue_header_format)
worksheet.merge_range('C6:C8', 'CANT', blue_header_format)	
worksheet.merge_range('D6:D8', 'DESCRIPCION', blue_header_format)	
worksheet.merge_range('E6:E8', 'CTO UNIT', blue_header_format)	
worksheet.merge_range('F6:F8', 'CTO TOTAL', blue_header_format)	
worksheet.merge_range('G6:G8', 'CALIBRE	', blue_header_format)
worksheet.merge_range('H6:H8', 'KG UNIT	', blue_header_format)
worksheet.merge_range('I6:I8', 'KG TOTAL', blue_header_format)	
worksheet.merge_range('J6:J8', 'CTO X KG', blue_header_format)	
worksheet.merge_range('K6:K8', 'M2 UNIT	', blue_header_format)
worksheet.merge_range('L6:L8', 'M2 TOT', blue_header_format)

row_count=9
def ret_na(value):
    try:
        x=float(value)
    except:
        x='NA'
    try:
        if(np.isnan(x)):
            x='NA'
        if(np.isinf(x)):
            x='NA'
    except:
        x='NA'
    return x
def num(value):
    try:
        x=float(value)
    except:
        x=0
    if((np.isnan(x))|(np.isinf(x))):
        x=0
    return x
#iterar sobre los productos

for i in range(0,len(products)):
    
    def my_func(row, table_name):
        return row in table_name
    piezas=materials.loc[materials['product'].apply(my_func,table_name=products['tabla'].values[i])]
    costo_product=products['cost'].values[i]
    n=len(piezas)
    
    print(n,products['tabla'].values[i],row_count,products['cost'].values[i])
    #Cosntruir la redaccion del producto
    if('joist' in products['tabla'].values[i]):
        peralte=' Peralte:'+str(products['camber'].values[i])
    else:
        peralte=''
    if(products[largo_cols].sum(axis=1).values[i]>0):
        largo='LARGO: '+str(products[largo_cols].sum(axis=1).values[i])+'M'
    else: largo=''
    if(products[ancho_cols].sum(axis=1).values[i]>0):
        ancho= 'ANCHO: '+str(products[ancho_cols].sum(axis=1).values[i])+'M'
    else: ancho=''
    print('GENERANDO LA DESCRIPCION::-----:::---:::---')
    print(largo,ancho,peralte)
    descripcion=tablas[products['tabla'].values[i]]+products['protector'].values[i]+' '+products['model'].values[i]+peralte+largo+ancho
    if(i%2==1):
        formato=blue_content
        formato_unit=blue_content_unit
    if(i%2==0):
        formato=blue_content_lite
        formato_unit=blue_content_unit_lite
    
    #pda
    worksheet.write('A'+str(row_count), str(i*n+1), formato)
    #sku
    worksheet.write('B'+str(row_count), products['sku'].values[i], formato)
    worksheet.write('C'+str(row_count), str(products['amount'].values[i]), formato_unit)
    #descripcion
    worksheet.write('D'+str(row_count), descripcion, formato)
    #costos
    print(costo_product)
    if(products['total_price'].values[i]>0):
        precio_total=products['total_price'].values[i]
        if(products['unit_price'].values[i]>0):
            precio_unitario=products['unit_price'].values[i]
        else:
             precio_unitario=precio_total/products['amount'].values[i]
    else:
        precio_total=products[price_cols].sum(axis=1, numeric_only=True)[i]
        precio_unitario=products['amount'].values[i]*products[price_cols].sum(axis=1, numeric_only=True)[i]
    worksheet.write('E'+str(row_count), ret_na(precio_unitario), formato)
    worksheet.write('F'+str(row_count), ret_na(precio_total), formato)
    #calibre
    worksheet.write('G'+str(row_count), str(ret_na(products['caliber'].values[i])), formato)
    #pesos
    worksheet.write('H'+str(row_count),(num(products['total_weight'].values[i])+num(products['total_kg'].values[i])+products['weight'].values[i]+products['weight_kg'].values[i])/products['amount'].values[i], formato_unit)
    worksheet.write('I'+str(row_count),(num(products['total_weight'].values[i])+num(products['total_kg'].values[i])+products['weight'].values[i]+products['weight_kg'].values[i]), formato_unit)
    try: 
        worksheet.write('J'+str(row_count),(num(products['amount'].values[i]*products['cost'].values[i])/(num(products['total_weight'].values[i])+num(products['total_kg'].values[i]))), formato)
    except:
        worksheet.write('J'+str(row_count),0, formato)
    #medidas
    worksheet.write('K'+str(row_count),products['m2'].values[i]+products['total_m2'].values[0], formato_unit)
    worksheet.write('L'+str(row_count),(products['m2'].values[i]+products['total_m2'].values[0])*products['amount'].values[i], formato_unit)
    row_count=row_count+1
    #PIEZAS PIEZAS PIEZAS CICLO DE PIEZAS
    for j in range(0,n):
        
        print('entre al ciclo')
        print(piezas['cost'].fillna(0).values[j],piezas['amount'])
        costo= piezas['cost'].fillna(0).values[j].sum()
        cant= piezas['amount'].fillna(0).values[j].sum()
        worksheet.write('A'+str(row_count), str(i*n+2+j), formato)
        #sku
        worksheet.write('B'+str(row_count), ''.join(materials['sku'].fillna('').values[0]), formato)
        worksheet.write('C'+str(row_count), str(piezas['amount'].values[j]), formato)
        worksheet.write('D'+str(row_count), str(piezas['description'].fillna('').values[j][0])+str(piezas['piece'].fillna('').values[j]), formato)
        #costos
        worksheet.write('E'+str(row_count),costo, formato)
        worksheet.write('F'+str(row_count), cant*costo, formato)
        #calibre
        worksheet.write('G'+str(row_count), piezas['type'].values[j][0]+piezas['type'].values[j][1], formato_unit)
        #pesos
        worksheet.write('H'+str(row_count),str(0.0), formato_unit)
        worksheet.write('I'+str(row_count),str(0.0), formato_unit)
        worksheet.write('J'+str(row_count), 0.0, formato)
        #medidas
        worksheet.write('K'+str(row_count), 0.0, formato_unit)
        worksheet.write('L'+str(row_count), 0.0, formato_unit)
        row_count=row_count+1
trow=row_count


#TOTALES
worksheet.merge_range('A'+str(trow+1)+':C'+str(trow), 'TOTAL (EQV M.N)', blue_header_format_bold)
worksheet.write_formula('F'+str(trow),'{=SUM(F9:F'+str(trow-1)+')}',blue_footer_format_bold)
worksheet.write_formula('I'+str(trow),'{=SUM(I9:I'+str(trow-1)+')}',blue_footer_format_bold_kg)
worksheet.write_formula('L'+str(trow),'{=SUM(L9:L'+str(trow-1)+')}',blue_footer_format_bold_kg)

# worksheet.write('K'+str(trow), str(cobros['amount'].sum()), blue_content)
# worksheet.write('L'+str(trow), str(cobros['exchange_sell'].values[0]*cobros['amount'].sum()), blue_content_bold)

#RESUMEN
worksheet.merge_range('B'+str(trow+3)+':C'+str(trow+4),'RESUMEN DE KILOS',blue_header_format_bold)
#subtabla 1, kilos
worksheet.write('B'+str(trow+5),'KILOS',blue_header_format)
worksheet.write('C'+str(trow+5),'CALIBRE',blue_header_format)
suma_peso=0
art_i=0
for i in products['caliber'].fillna('NA').astype(str).unique():
    if(i!='NA'):
        print(i)
        p=products.loc[products['caliber']==i]
        sum_kg=p['weight'].fillna(0).sum()+p['total_weight'].fillna(0).sum()+p['weight_kg'].fillna(0).sum()+p['total_kg'].fillna(0).sum()
        suma_peso=suma_peso+sum_kg
        worksheet.write('B'+str(trow+6+art_i),sum_kg,blue_content)
        worksheet.write('C'+str(trow+6+art_i),i,blue_content)
        art_i=art_i+1

worksheet.write('B'+str(trow+5+len(products['caliber'].unique())),suma_peso,blue_footer_format_bold_kg)
#subtabla2 costos F-G-H-I-J-K-L-M-N
#                     F-G-H-I-J-K-L-M-N
worksheet.merge_range('F'+str(trow+4)+':I'+str(trow+4),'RESUMEN DE COSTOS',blue_header_format_bold)
worksheet.write('J'+str(trow+4),'POSICION',blue_header_format)
worksheet.write('K'+str(trow+4),'KILOS',blue_header_format)
worksheet.write('L'+str(trow+4),'PORCENTAJE',blue_header_format)

worksheet.merge_range('F'+str(trow+5)+':I'+str(trow+5),'CONTRATO UNITARIO SOLO MATERIALES',blue_header_format)
worksheet.merge_range('F'+str(trow+6)+':I'+str(trow+6),'CONTRATO UNITARIO SOLO ARMADO',blue_header_format)
worksheet.merge_range('F'+str(trow+7)+':I'+str(trow+7),'CONTRATO UNITARIO SOLO TRASLADO',blue_header_format)
worksheet.merge_range('F'+str(trow+8)+':I'+str(trow+8),'CONTRATO UNITARIO COMBINADO',blue_header_format)


costo_total=products['cost'].sum()+materials['cost'].fillna(0).sum(axis=1, numeric_only=True).sum()

worksheet.write('J'+str(trow+5),materials['cost'].fillna(0).sum(axis=1, numeric_only=True).sum(),blue_content)
worksheet.write('K'+str(trow+5),0,blue_content_unit)
worksheet.write('L'+str(trow+5),materials['cost'].fillna(0).sum(axis=1, numeric_only=True).sum()/costo_total*100,blue_content_unit)

worksheet.write('J'+str(trow+6),products.loc[products['tabla'].isin(['quotation_installs','quotation_uninstalls']),'cost'].sum(),blue_content)
worksheet.write('K'+str(trow+6),products.loc[products['tabla'].isin(['quotation_installs','quotation_uninstalls']),'cost'].sum()/costo_total*100,blue_content_unit)
worksheet.write('L'+str(trow+6),0,blue_content_unit)

worksheet.write('J'+str(trow+7),products.loc[(products['tabla'].isin(['quotation_travel_assignments','packagings'])),'cost'].sum(),blue_content)
worksheet.write('K'+str(trow+7),0,blue_content_unit)
worksheet.write('L'+str(trow+7),products.loc[(products['tabla'].isin(['quotation_travel_assignments','packagings'])),'cost'].sum()/costo_total*100,blue_content_unit)

worksheet.write('J'+str(trow+8),products['cost'].sum(),blue_content)
worksheet.write('K'+str(trow+8),products[cols_kg].sum(axis=1, numeric_only=True).sum(),blue_content_unit)
worksheet.write('L'+str(trow+8),'100%',blue_content)
#TODO: calcular bien esto, to6al menos iva


worksheet.set_column('B:B',20)
worksheet.set_column('D:D',30)
worksheet.set_column('J:J',15)
worksheet.set_column('E:E',15)
worksheet.set_column('F:F',15)
worksheet.set_column('G:L',15)
worksheet.set_column('N:R',15)
#worksheet.set_landscape()
worksheet.set_paper(9)
worksheet.fit_to_pages(1, 1)  
workbook.close()

c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\fgonzalez\AppData\Local\Temp\ipykernel_6532\855393210.py:2

double_deep_crossbars
double_deep_floors
double_deep_floor_reinforcements
double_deep_heavy_load_frames
double_deep_joist_box25s
double_deep_joist_box25_caliber14s
double_deep_joist_box2s
double_deep_joist_box2_caliber14s
double_deep_joist_c2_s
double_deep_joist_chairs
double_deep_joist_l25_caliber14s
double_deep_joist_l25_s
double_deep_joist_l2_caliber14s
double_deep_joist_l2_s
double_deep_joist_lrs
double_deep_joist_structurals
double_deep_miniature_frames
double_deep_spacers
double_deep_structural_frames
freights
grills
installations
packagings
quot25_j_galvanized_panels
quot25_j_painted_panels
quot2_j_galvanized_panels
quot2_j_painted_panels
quotation_administratives
quotation_installs
quotation_protectors
quotation_specials
quotation_escuadras
quotation_gangplank_angles
quotation_respaldos
quotation_travel_assignments


C:\Users\fgonzalez\AppData\Local\Temp\ipykernel_6532\855393210.py:283: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  products=products.append(p,ignore_index=True)
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\fgonzalez\AppData\Local\Temp\ipykernel_6532\855393210.py:283: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  products=products.append(p,ignore_index=True)
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase 

quotation_uninstalls
quot_chair_j_galvanized_panels
quot_chair_j_painted_panels
selective_crossbars
selective_floors
selective_floor_reinforcements
selective_heavy_load_frames
14
selective_heavy_load_frames
selective_joist_box25s
selective_joist_box25_caliber14s
selective_joist_box2s
selective_joist_box2_caliber14s
selective_joist_c2_s
selective_joist_chairs
selective_joist_l25_caliber14s
14
selective_joist_l25_caliber14s
selective_joist_l25_s
selective_joist_l2_caliber14s
selective_joist_l2_s
selective_joist_lrs
selective_joist_structurals
selective_miniature_frames
selective_spacers
12
selective_spacers
selective_structural_frames
uninstalls


C:\Users\fgonzalez\AppData\Local\Temp\ipykernel_6532\855393210.py:283: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  products=products.append(p,ignore_index=True)
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\fgonzalez\AppData\Local\Temp\ipykernel_6532\855393210.py:283: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  products=products.append(p,ignore_index=True)
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase 

wood
quotation_drive_in_soportes
quotation_drive_in_guias
quotation_drive_in_arriostrados
quotation_drive_in_brazos
quotation_galletas
0 packagings 9 3.02
GENERANDO LA DESCRIPCION::-----:::---:::---
  
3.02
0 quotation_installs 10 273.52
GENERANDO LA DESCRIPCION::-----:::---:::---
  
273.52
3 selective_heavy_load_frames 11 148525.19999999998
GENERANDO LA DESCRIPCION::-----:::---:::---
 1.06 
148525.19999999998
entre al ciclo
[1. 0.] 0    4
1    4
2    4
Name: amount, dtype: int64
entre al ciclo
[8. 0.] 0    4
1    4
2    4
Name: amount, dtype: int64
entre al ciclo
[ 0.  26.5] 0    4
1    4
2    4
Name: amount, dtype: int64
1 selective_joist_l25_caliber14s 15 264.4974
GENERANDO LA DESCRIPCION::-----:::---:::---
Largo4.88   Peralte:4.0
264.4974
entre al ciclo
[1. 0.] 5    2
Name: amount, dtype: int64
2 selective_spacers 17 6.1425
GENERANDO LA DESCRIPCION::-----:::---:::---
Largo0.35  
6.1425
entre al ciclo
[1. 0.] 16    4
28    4
Name: amount, dtype: int64
entre al ciclo
[1. 0.] 16    4


In [ ]:
0.3

In [25]:
4000/3

1333.3333333333333

In [22]:
products

,id,quotation_id,amount,type,depth,developing,long,caliber,kg_m2,weight,...,cantidad,uncut_front,cut_front,uncut_background,cut_background,desarrollo,piezas_nec,piece_weight,development,peso
0,29,192,324,NaN,0.91,NaN,NaN,14,NaN,0,...,324.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8,192,3,NaN,NaN,0.084,0.3,12,21.25,0,...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,192,3,NaN,NaN,NaN,NaN,NaN,NaN,0,...,1.0,3.0,3.0,3.0,3.0,NaN,NaN,NaN,NaN,NaN


In [2]:

redact={
'double_deep_crossbars' : 'VIGA TIPO CAJA DE DOBLE PROFUNDIDAD DE 2.5',
'double_deep_floors' : 'SUELOS DE DOBLE PROFUNDIDAD',
'double_deep_floor_reinforcements' : 'REFUERZOS PARA EL SUELO DE DOBLE PROFUNDIDAD',
'double_deep_heavy_load_frames' : 'BASTIDORES DE CARGA PESADA DE DOBLE PROFUNDIDAD',
'double_deep_joist_box25s' : 'VIGA TIPO CAJA DE DOBLE PROFUNDIDAD DE 2.5',
'double_deep_joist_box25_caliber14s' : 'VIGA TIPO CAJA DE DOBLE PROFUNDIDAD DE 2.5 CALIBRE 14',
'double_deep_joist_box2s' : 'VIGA TIPO CAJA DE DOBLE PROFUNDIDAD DE 2',
'double_deep_joist_box2_caliber14s' : 'VIGA TIPO CAJA DE DOBLE PROFUNDIDAD DE 2 CALIBRE 14',
'double_deep_joist_c2_s' : 'VIGA DE DOBLE PROFUNDIAD TIPO C2',
'double_deep_joist_chairs' : 'VIGA DE DOBLE PROFUNDIAD TIPO SILLA',
'double_deep_joist_l25_caliber14s' : 'VIGA TIPO L DE DOBLE PROFUNDIDAD DE 2.5 CALIBRE 14',
'double_deep_joist_l25_s' : 'VIGA TIPO L DE DOBLE PROFUNDIDAD DE 2.5 ',
'double_deep_joist_l2_caliber14s' : 'VIGA TIPO L DE DOBLE PROFUNDIDAD DE 2 CALIBRE 14',
'double_deep_joist_l2_s' : 'VIGA TIPO L DE DOBLE PROFUNDIDAD DE 2 ',
'double_deep_joist_lrs' : 'VIGA DE DOBLE PROFUNDIAD TIPO LR',
'double_deep_joist_structurals' : 'VIGA DE DOBLE PROFUNDIAD TIPO ESTRUCTURAL',
'double_deep_miniature_frames' : 'MARCOS EN MINIATURA DE DOBLE PROFUNDIDAD',
'double_deep_spacers' : 'ESPACIADOR DE DOBLE PROFUNDIDAD',
'double_deep_structural_frames' : 'MARCOS ESTRUCTURALES DE DOBLE PROFUNDIDAD',
'freights' : 'FLETES',
'grills' : 'Parrilla Métalica',
'installations' : 'INSTALACION',
'packagings' : 'FLETE',
'quot25_j_galvanized_panels' : 'Panel metálico Galvanizado modelo ',
'quot25_j_painted_panels' : 'Panel metálico Pintado modelo',
'quot2_j_galvanized_panels' : 'Panel metálico Galvanizado modelo',
'quot2_j_painted_panels' : 'Panel metálico Pintado modelo',
'quotation_administratives' : 'ADMINISTRATIVO',
'quotation_installs' : 'INSTALACION',
'quotation_protectors' : 'Protector ',

'quotation_escuadras' : 'ESCUADRA DE SOPORTE',
'quotation_gangplank_angles' : 'ANGULO RANURADO',
'quotation_respaldos':'RESPALDO ',
'quotation_specials' : 'ESPECIAL',
'quotation_travel_assignments' : 'VIATICOS ',
'quotation_uninstalls' : 'DESINTALACION',
'quot_chair_j_galvanized_panels' : 'Panel métalico galvanizado tipo silla',
'quot_chair_j_painted_panels' : 'Panel Métalico Pintado tipo silla',
'selective_crossbars' : 'Crosbaar metálico galvanizado, modelo ',
'selective_floors' : 'Piso Metálico modelo ',
'selective_floor_reinforcements' : 'Refuerzos para Piso Métalico modelo ',
'selective_heavy_load_frames' : 'Marco de Rack Pesado, con poste ',
'selective_joist_box25s' : 'Viga tipo caja de 2.5 modelo ',
'selective_joist_box25_caliber14s' : 'Viga tipo caja de 2.5 calibre 14 modelo',
'selective_joist_box2s' : 'Viga tipo caja de 2.0 modelo',
'selective_joist_box2_caliber14s' : 'viga tipo caja de 2.0 calibre14',
'selective_joist_c2_s' : 'Viga tipo C2 modelo',
'selective_joist_chairs' : 'Viga tipo silla modelo',
'selective_joist_l25_caliber14s' : 'Viga tipo l25 calibre 14 modelo',
'selective_joist_l25_s' : 'Viga tipo L25 modelo',
'selective_joist_l2_caliber14s' : 'Viga tipo L2 calibre14 modelo',
'selective_joist_l2_s' : 'Viga tipo L2 modelo',
'selective_joist_lrs' : 'Viga tipo LR modelo',
'selective_joist_structurals' : 'Viga tipo estructural modelo ',
'selective_miniature_frames' : 'MARCOS EN MINIATURA',
'selective_spacers' : 'Distanciador modelo',
'selective_structural_frames' : 'Marco de Rack Estructu ral con poste ',
'uninstalls' : 'DESINSTALACION',
'wood' : 'Panel de Madera',

'quotation_drive_in_soportes' : 'Soportes para tarimas',
'quotation_drive_in_guias' : 'Guias para montacargas',
'quotation_drive_in_arriostrados' : 'Arriostrados tipo "L" modelo',
'quotation_drive_in_brazos' : 'Brazos para drive in modelo',
'quotation_galletas':'Galletas'
}


In [17]:
for i in redact :
    
    # print("alter table "+str(i)+" add column cart_id int;")
    print("delete from "+str(i)+" ;")

delete from double_deep_crossbars ;
delete from double_deep_floors ;
delete from double_deep_floor_reinforcements ;
delete from double_deep_heavy_load_frames ;
delete from double_deep_joist_box25s ;
delete from double_deep_joist_box25_caliber14s ;
delete from double_deep_joist_box2s ;
delete from double_deep_joist_box2_caliber14s ;
delete from double_deep_joist_c2_s ;
delete from double_deep_joist_chairs ;
delete from double_deep_joist_l25_caliber14s ;
delete from double_deep_joist_l25_s ;
delete from double_deep_joist_l2_caliber14s ;
delete from double_deep_joist_l2_s ;
delete from double_deep_joist_lrs ;
delete from double_deep_joist_structurals ;
delete from double_deep_miniature_frames ;
delete from double_deep_spacers ;
delete from double_deep_structural_frames ;
delete from freights ;
delete from grills ;
delete from installations ;
delete from packagings ;
delete from quot25_j_galvanized_panels ;
delete from quot25_j_painted_panels ;
delete from quot2_j_galvanized_panels ;
delet

In [8]:
factores.loc[factores['caliber']=='14']

,id,description,caliber,type,system,piece,sku,cost,steel,f_vta,f_desp,f_emb,f_desc,f_total,created_at,updated_at
0,1,LAMINA NEGRA RC,14,Negra,ESTANTERIA,ANGULO,None,30.0,1,2.3,2.0,1.015,0.94,4.915,2023-03-17 00:00:00,2024-07-11 03:30:32
10,11,LAMINA GALVANIZADA RC,14,Galvanizada,SELECTIVO,MARCO,None,25.0,1,2.3,2.0,1.015,0.95,4.915,2023-03-17 00:00:00,2024-07-11 06:10:25
15,16,LAMINA NEGRA RC,14,Negra,SELECTIVO,VIGA,None,30.0,1,2.3,2.0,1.015,0.94,4.967,2023-03-17 00:00:00,2024-07-11 03:30:32
22,23,LAMINA NEGRA RC,14,Negra,MINI RACK,MARCO,None,30.0,1,2.3,2.0,1.015,0.94,4.967,2023-03-17 00:00:00,2024-07-11 03:30:32
25,26,LAMINA GALVANIZADA RC,14,Galvanizada,ACCESORIOS,CROSS BAR,None,25.0,1,2.0,2.0,1.015,0.94,4.319,2023-03-17 00:00:00,2024-07-20 05:56:41
30,31,LAMINA NEGRA RC,14,Negra,ACCESORIOS,PANELES,None,30.0,1,2.3,2.0,1.015,0.94,4.967,2023-03-17 00:00:00,2024-07-11 03:30:32
31,32,LAMINA GALVANIZADA RC,14,Galvanizada,ACCESORIOS,PANELES,None,25.0,1,2.0,2.0,1.015,0.94,4.319,2023-03-17 00:00:00,2024-07-11 06:10:25
38,39,LAMINA NEGRA RC,14,Negra,ACCESORIOS,PISO,None,30.0,1,2.3,2.0,1.015,0.94,4.967,2023-03-17 00:00:00,2024-07-11 03:30:32
39,40,LAMINA GALVANIZADA RC,14,Galvanizada,ACCESORIOS,PISO,None,25.0,1,2.0,2.0,1.015,0.94,4.319,2023-03-17 00:00:00,2024-07-11 06:10:25
40,41,LAMINA GALVANIZADA RC,14,Galvanizada,ACCESORIOS,CALZAS,TC0000000,25.0,1,3.0,2.0,1.015,0.95,4.915,2023-03-17 00:00:00,2024-07-11 06:10:25


In [10]:
for i in range(len(loading)):
    cursor.execute(query,tuple([loading['calibre'].values[i],
                                loading['capacidad'].values[i],
                                loading['longitud'].values[i],
                                loading['peralte'].values[i],
                                   ]))
cnx.commit()

In [1]:
'SELECTIVO'

'Selectivo'

In [11]:
lr=pd.read_excel('tablas_lr.xlsx')

In [16]:
lr=lr.astype(str)

In [18]:
lr.columns

Index(['largo', 'sku', 'peso', 'm2', 'precio', 'peralte', 'calibre'], dtype='object')

In [24]:
query='INSERT INTO type_l_r_joists(length,sku,weight,m2,price,camber,caliber) values(%s,%s,%s,%s,%s,%s,%s)'

In [26]:
for i in range(len(lr)):
    cursor.execute(query,tuple([lr['largo'].values[i],
                                lr['sku'].values[i],
                                lr['peso'].values[i],
                                lr['m2'].values[i],
                                lr['precio'].values[i],   
                                lr['peralte'].values[i],
                                lr['calibre'].values[i],
                                   ]))
cnx.commit()

In [56]:
data=pd.read_excel('entpint.xlsx',sheet_name='ENTREPAÑO C-20',skiprows=[0,2])

In [57]:
data.columns

Index(['DFONDO M', 'DFRENTE M', 'VFONDO M', 'VLARGO M', 'SKU', 'CALIBRE',
       'KG / M2', 'PESO ', 'M2 ', 'PRECIO VENTA', 'CAPACIDAD DE CARGA (KG)',
       'CON 1 REFUERZO', 'CON 2 REFUERZOS'],
      dtype='object')

In [59]:
query='INSERT INTO estanteria_entrepanios(development_deep,development_front,deep,length,sku,caliber,kgm2,weight,m2,loading_capacity,reforcement1,reforcement2,type) values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'

In [65]:
for caliber in ['20','22','24','26']:
    data=pd.read_excel('entgal.xlsx',sheet_name='ENTREPAÑO C-'+caliber,skiprows=[0,2])
    for i in range(len(data)):
        cursor.execute(query,tuple([
        str(data['DFONDO M'].values[i]),
        str(data['DFRENTE M'].values[i]),
        str(data['VFONDO M'].values[i]),
        str(data['VLARGO M'].values[i]),
        str(data['SKU'].values[i]),
        caliber,
        str(data['KG / M2'].values[i]),
        str(data['PESO '].values[i]),
        str(data['M2 '].values[i]),
        str(data['CAPACIDAD DE CARGA (KG)'].values[i]),
        str(data['CON 1 REFUERZO'].values[i]),
        str(data['CON 2 REFUERZOS'].values[i]),
        'GALVANIZADO'
                  ]))
cnx.commit()

In [28]:
for i in data.columns:


    
    print("str(data['"+i+"'].values[i]),")

str(data['DFONDO M'].values[i]),
str(data['DFRENTE M'].values[i]),
str(data['VFONDO M'].values[i]),
str(data['VLARGO M'].values[i]),
str(data['SKU'].values[i]),
str(data['CALIBRE'].values[i]),
str(data['KG / M2'].values[i]),
str(data['PESO '].values[i]),
str(data['M2 '].values[i]),
str(data['PRECIO VENTA'].values[i]),
str(data['CAPACIDAD DE CARGA (KG)'].values[i]),
str(data['CON 1 REFUERZO'].values[i]),
str(data['CON 2 REFUERZOS'].values[i]),


In [61]:
cursor.execute('truncate table estanteria_entrepanios')
cursor.execute('alter table estanteria_entrepanios AUTO_INCREMENT = 1')

In [53]:
cnx.commit()

In [66]:
e=pd.read_sql('select * from estanteria_entrepanios',cnx)

C:\Users\hp\AppData\Local\Temp\ipykernel_6452\401490957.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  e=pd.read_sql('select * from estanteria_entrepanios',cnx)


In [68]:
e['type'].unique()

array(['PINTADO', 'GALVANIZADO'], dtype=object)

In [11]:
data=pd.read_excel('angulos.xlsx',sheet_name='ANGULO RANURADO C-14',skiprows=[0,1,2,])

In [7]:
query='INSERT INTO gangplank_angles(length,sku,caliber,weight,m2,deep) values(%s,%s,%s,%s,%s,%s)'

In [10]:
for i in range(len(data)):
        cursor.execute(query,tuple([
        str(data['Unnamed: 0'].values[i]),
        str(data['SKU'].values[i]),
        '14',
        str(data['PESO (KG).1'].values[i]),
        str(data['M2.1'].values[i]),
        '0.76'
                  ]))
cnx.commit()

In [4]:
data.columns

Index(['Unnamed: 0', 'SKU', 'PESO (KG)', 'M2', 'PRECIO', 'SKU.1',
       'PESO (KG).1', 'M2.1', 'PRECIO.1'],
      dtype='object')

In [12]:
data

,Unnamed: 0,SKU,PESO (KG),M2,PRECIO,SKU.1,PESO (KG).1,M2.1,PRECIO.1
0,0.01,TC0000123991,0.01098,0.00144,1.540667,TC0000124357,0.01403,0.00184,1.968631
1,0.02,TC0000123992,0.02196,0.00288,2.981937,TC0000124358,0.02806,0.00368,3.810253
2,0.03,TC0000123993,0.03294,0.00432,4.472905,TC0000124359,0.04209,0.00552,5.715379
3,0.04,TC0000123994,0.04392,0.00576,5.963874,TC0000124360,0.05612,0.00736,7.620505
4,0.05,TC0000123995,0.05490,0.00720,7.454842,TC0000124361,0.07015,0.00920,9.525632
...,...,...,...,...,...,...,...,...,...
361,3.62,TC0000124352,3.97476,0.52128,539.730568,TC0000124718,5.07886,0.66608,689.655726
362,3.63,TC0000124353,3.98574,0.52272,541.221537,TC0000124719,5.09289,0.66792,691.560853
363,3.64,TC0000124354,3.99672,0.52416,542.712505,TC0000124720,5.10692,0.66976,693.465979
364,3.65,TC0000124355,4.00770,0.52560,544.203474,TC0000124721,5.12095,0.67160,695.371105


In [14]:
data=pd.read_excel('respaldos.xlsx',skiprows=[0,2],sheet_name='RESPALDOS C-24')

In [19]:
query='INSERT INTO respaldos(deep,front,sku,caliber,kgm2,weight,m2) values(%s,%s,%s,%s,%s,%s,%s)'

In [22]:
for i in range(len(data)):
        cursor.execute(query,tuple([
        str(data['DESARROLLO'].values[i]),
        str(data['Unnamed: 2'].values[i]),
        str(data['SKU'].values[i]),
        str(data['CALIBRE'].values[i]),
        str(data['KG / M2'].values[i]),
        str(data['PESO '].values[i]),
        str(data['M2 '].values[i]),
                  ]))
cnx.commit()

In [16]:
data.columns

Index(['Unnamed: 0', 'DESARROLLO', 'Unnamed: 2', 'SKU', 'CALIBRE', 'KG / M2',
       'PESO ', 'M2 ', 'PRECIO VENTA'],
      dtype='object')